In [4]:
from ipywidgets import widgets, AppLayout, interact, interactive, fixed
from IPython.display import display, Math
import numpy as np
import src.bings_model as bm 
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
sns.set_context('notebook')

def plot_process(bups, a, params):
    fig, ax = plt.subplots(4,1, sharex=True, 
                           figsize=(6,7),
                           gridspec_kw={'height_ratios': [.2, .2, .45, 1]})
    
    bm.plot_clicktrain(bups, ax=ax[0])
    ax[0].set_ylabel('')
    ax[0].spines[['left','bottom']].set_linewidth(0)
    ax[0].axes.get_xaxis().set_visible(False)
    ax[0].tick_params(left=False)
        
    bm.plot_adaptation_process(bups, ax=ax[1])
    ax[1].set_xlabel('')
    ax[1].axes.get_xaxis().set_visible(False)
    ax[1].spines[['left']].set_linewidth(0)
    
    bm.plot_adapted_clicks(bups, ax=ax[2])
    ax[2].axes.get_xaxis().set_visible(False)
    ax[2].spines[['bottom']].set_visible(False)
    ax[2].spines[['left']].set_linewidth(0)
    ax[2].spines[['bottom']].set_linewidth(.5)

    
    bm.plot_accumulation(bups, a, params, ax=ax[3])
    
    fig.tight_layout()
    ax[0].set_xlim([-.025, bups['duration']+.025])
    #fig.align_ylabels()
    plt.show()
    return None

def clicks_eventhandler(change):
    global bups
    bups = bm.make_clicktrain(total_rate=rate_slider.value, gamma=gamma_slider.value, 
                               duration=dur_slider.value, rng=seed_slider.value)
    adaptation_eventhandler(change)

    
def adaptation_eventhandler(change):
    bm.make_adapted_clicks(bups, phi=phi_slider.value, tau_phi=tau_phi_slider.value)  
    integration_eventhandler(change)
    
def integration_eventhandler(change):
    global a, params
    a, params = bm.integrate_adapted_clicks(bups, lam=lambda_slider.value, s2s=s2s_slider.value,
                             s2a=s2a_slider.value, s2i=s2i_slider.value, bias=bias_slider.value,
                             B=B_slider.value, nagents=nagent_slider.value, rng=seed_slider.value)
    choice_eventhandler(change)
    
def choice_eventhandler(change):
    plot_out.clear_output(wait=True)
    
    with plot_out:
        plot_process(bups, a, params)
        

plot_out = widgets.Output()

bups_text = widgets.Output()
with bups_text:
    print("Click train parameters\n")
    
adaptation_text = widgets.Output()
with adaptation_text:
    print("\nAdaptation parameters\n")
    #display(Math(r'$dC = \frac{1-C}{\tau_{\phi}}dt + (\phi - 1) C (\delta_{t,t_R} + \delta_{t,t_L})$'))
    
integration_text = widgets.Output()
with integration_text:
    print("\n\nIntegration parameters \n")
    #display(Math(r'$da = (\eta C \delta_R - \eta C \delta_L)dt + \lambda a dt + \sigma_adW $'))
    #display(Math('$a_0 = \mathcal{N}(0, \sigma_i^2)$'))
    
choice_text = widgets.Output()
with choice_text:
    print("\nChoice parameters\n")

# Define click parameters
seed_slider = widgets.IntSlider(value=1, min=0, max=10, description="Seed")
rate_slider = widgets.IntSlider(value=40,min=5,max=50,step=5,description=r'$r_L+r_R$ (Hz)')
gamma_slider = widgets.FloatSlider(value=1.5,min=-5,max=5,step=.25,description=r"$\gamma = \log \frac{r_1}{r_2}$")
dur_slider = widgets.FloatSlider(value=1,min=.25,max=5,step=.25,description="T (s)")
        
seed_slider.observe(clicks_eventhandler, names='value')
rate_slider.observe(clicks_eventhandler, names='value')
gamma_slider.observe(clicks_eventhandler, names='value')
dur_slider.observe(clicks_eventhandler, names='value')

# Define adaptation parameters
phi_slider = widgets.FloatSlider(value=.1,min=.001,max=1,step=.05,description=r"$\phi$")
tau_phi_slider = widgets.FloatSlider(value=.15,min=.001,max=1,step=.05,description=r"$\tau_{\phi}$")

phi_slider.observe(clicks_eventhandler, names='value')
tau_phi_slider.observe(clicks_eventhandler, names='value')

# Define integration parameters
lambda_slider = widgets.FloatSlider(value=0., min = -5., max=5., step=.25, description=r"$\lambda$")
s2s_slider = widgets.FloatSlider(value=1., min = .001, max=50., step=.25, description=r"$\sigma^2_s$")
s2a_slider = widgets.FloatSlider(value=.05, min = .001, max=10., step=.25, description=r"$\sigma^2_a$")
s2i_slider = widgets.FloatSlider(value=.5, min = .001, max=5., step=.25, description=r"$\sigma^2_i$")
B_slider = widgets.FloatSlider(value=10., min = 0., max=25., step=1, description=r"$B$")
nagent_slider = widgets.IntSlider(value=10, min = 1, max=50, description=r"N samples")

lambda_slider.observe(clicks_eventhandler, names='value')
s2s_slider.observe(clicks_eventhandler, names='value')
s2a_slider.observe(clicks_eventhandler, names='value')
s2i_slider.observe(clicks_eventhandler, names='value')
B_slider.observe(clicks_eventhandler, names='value')
nagent_slider.observe(clicks_eventhandler, names='value')

# Define choice parameters
bias_slider = widgets.FloatSlider(value=0, min = -5, max=5., step=.05, description=r"Bias")
lapse_slider = widgets.FloatSlider(value=.05, min = 0., max=1., step=.01, description=r"Lapse")
bias_slider.observe(clicks_eventhandler, names='value')
lapse_slider.observe(clicks_eventhandler, names='value')

bup_inputs = widgets.VBox([bups_text, seed_slider, rate_slider, gamma_slider, dur_slider])
adaptation_inputs = widgets.VBox([adaptation_text, phi_slider, tau_phi_slider])
integration_inputs = widgets.VBox([integration_text, lambda_slider, s2s_slider, s2a_slider, 
                                   s2i_slider, B_slider, nagent_slider])
choice_inputs = widgets.VBox([choice_text, bias_slider, lapse_slider])
inputs = widgets.VBox([bup_inputs, adaptation_inputs, integration_inputs, choice_inputs])

# Create GUI

In [14]:
interface = widgets.HBox([inputs, plot_out])
display(interface)
clicks_eventhandler([])

# Generate click trains
The left and right Poisson Click trains, $\delta_{t,t_L}$ and $\delta_{t,t_R}$ are generated by sampling at two different Poisson rates $r_L$ and $r_R$. 

These generative click rates are determined by setting the overall total rate (default value is 40Hz) and the contrast parameter $\gamma = \log \frac{r_R}{r_L}$.

# Integrate the adapted clicks
Now, we need to integrate the adapted clicks. We'll also apply per-click noise (aka sensory noise) with standard deviation $\sigma_s$, memory noise with s.d.=$\sigma_a$, initial noise with s.d.=$\sigma_i$, and an integration time constant $\lambda$. This gives us our accumulation value
\begin{align}
da &= (\eta C \delta_R - \eta C \delta_L)dt + \lambda a dt + \sigma_adW , & a_0 = \mathcal{N}(0, \sigma_i^2)
\end{align}

# To Do List
- [x] make bups sync
- [x] make bups sync with accumulation values
- [x] depict lapse/choices
- [x] make plot autogenerate 
- [x] make bups a global so that we don't have to regenerate the clicks everytime something is updated
- [x] Separate src code from notebook

- [ ] make tests of adaptation process
- [ ] fix adaptation for positive values of phi
- [ ] make tests for accumulation values (compare to matlab results)

- [ ] Separate the integration parameters from one another (e.g., changing B shouldn't require recomputing the memory noise)
- [x] Align plots better
- [ ] Add description of computations

- [ ] look at implementation of per click noise (i remember there being something wrong with it)
- [ ] put it online using sphinx or something? # 
- [ ] link to bings rats' parameters and you can click a rat to get its parameters

Here's the link I used to learn to make these [how to](https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916)